<a href="https://colab.research.google.com/github/brunogriep/etm-nas/blob/main/SaveResults.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# create at 07/03/2022
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%%capture
!git clone https://github.com/google-research/nasbench
!pip install ./nasbench
!pip install xautodl

!pip install nats-bench
!pip install pyyaml==5.4.1
!pip install simplejson

In [3]:
%tensorflow_version 1.x


TensorFlow 1.x selected.


In [4]:
!git clone https://brunogriep@bitbucket.org/brunogriep/nas-wot.git

Cloning into 'nas-wot'...
remote: Enumerating objects: 352, done.
remote: Counting objects: 100% (352/352), done.
remote: Compressing objects: 100% (232/232), done.
remote: Total 352 (delta 168), reused 277 (delta 116), pack-reused 0
Receiving objects: 100% (352/352), 931.17 KiB | 3.55 MiB/s, done.
Resolving deltas: 100% (168/168), done.


In [5]:
%cd /content/nas-wot

/content/nas-wot


In [13]:
%%capture
'''
NATS-Bench for Topology and Size
'''
!tar xvf /content/drive/MyDrive/Mestrado/Codes/NATS-Bench/NATS-tss-v1_0-3ffb9-simple.tar -C /content/.
!tar xvf /content/drive/MyDrive/Mestrado/Codes/NATS-Bench/NATS-sss-v1_0-50262-simple.tar -C /content/.

'''
CIFAR10, CIFAR100, ImageNet16 datasets
# '''
# !cp -a /content/drive/MyDrive/Mestrado/Codes/cifar.python/cifar-10-batches-py/. /content/nas-wot/datasets/cifar10
# !cp -a /content/drive/MyDrive/Mestrado/Codes/cifar.python/cifar-100-python/. /content/nas-wot/datasets/cifar100
# !cp -a /content/drive/MyDrive/Mestrado/Codes/cifar.python/ImageNet16/. /content/nas-wot/datasets/ImageNet16

In [36]:
# imports 
import numpy as np
from nasbench import api
import random 

import torch
import torchvision
import torchvision.transforms as transforms
import argparse
import datasets
import nasspace
import os
import pandas as pd
import time
from scores import get_score_func
from scipy import stats
from pycls.models.nas.nas import Cell
from os import walk
from utils import add_dropout, init_network

In [37]:
parser = argparse.ArgumentParser(description='NAS Without Training')
parser.add_argument('--api_loc', default='/content/NATS-sss-v1_0-50262-simple/', type=str, help='path to API')
parser.add_argument('--augtype', default='none', type=str, help='which perturbations to use')
parser.add_argument('--batch_size', default=128, type=int)
parser.add_argument('--data_loc', default='./datasets/cifar10',type=str,help='dataset folder')
parser.add_argument('--dataset', default='cifar10', type=str)
parser.add_argument('--dropout', action='store_true')
parser.add_argument('--GPU', default='0', type=str)
parser.add_argument('--init', default='', type=str)
parser.add_argument('--maxofn', default=1, type=int, help='score is the max of this many evaluations of the network')
parser.add_argument('--n_runs', default=100, type=int)
parser.add_argument('--n_samples', default=1000, type=int)
parser.add_argument('--nasspace', default='nasbench201',type=str,help='the nas search space to use')
parser.add_argument('--num_iter', default=10, type=int)
parser.add_argument('--num_labels', default=1,  type=int,help='#classes (nasbench101)')
parser.add_argument('--num_modules_per_stack', default=3, type=int,help='#modules per stack (nasbench101)')
parser.add_argument('--num_stacks', default=3,  type=int, help='#stacks of modules (nasbench101)')
parser.add_argument('--repeat', default=1,type=int, help='how often to repeat a single image with a batch')
parser.add_argument('--save_loc', default='results',type=str, help='folder to save results')
parser.add_argument('--save_string', default='naswot', type=str, help='prefix of results file')
parser.add_argument('--score_type', default='log/mean', type=str)
parser.add_argument('--score', default='hook_trace', type=str, help='the score to evaluate')
parser.add_argument('--seed', default=1, type=int)
parser.add_argument('--sigma', default=0.05 ,type=float, help='noise level if augtype is "gaussnoise"')
parser.add_argument('--stem_out_channels', default=16, type=int, help='output channels of stem convolution (nasbench101)')
parser.add_argument('--trainval', action='store_true')

args = parser.parse_args(args=[])
os.environ['CUDA_VISIBLE_DEVICES'] = args.GPU

# Reproducibility
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
random.seed(args.seed)
np.random.seed(args.seed)
torch.manual_seed(args.seed)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


In [41]:
if args.dataset == 'cifar10':
  acc_type = 'ori-test'
  val_acc_type = 'x-valid'
else:
  acc_type = 'x-test'
  val_acc_type = 'x-valid'
args.trainval = True

In [53]:
month = "march"
mypath = f"/content/drive/MyDrive/Mestrado/2022/{month}-experiments/"
num_precision = 3
filenames = next(walk(mypath), (None, None, []))[2]
files = [k for k in filenames if 'random' in k]

df_test = pd.DataFrame()

for file in files:
  if "cifar10" in file: 
    args.data_loc = "./datasets/cifar10"
    args.dataset = "cifar10"
  elif 'cifar100' in file: 
    args.data_loc = "./datasets/cifar100"
    args.dataset = "cifar100"
  else: 
    args.dataset = "ImageNet16-120"
    args.data_loc = "./datasets/ImageNet16"

  searchspace = nasspace.get_search_space(args)

  results = pd.read_csv(mypath + file, sep='\t', encoding='utf-8')
  accs = results["accs"]
  chosen = results["chosen"]
  scores = results["topscores"]
  times = results["times"]
  val_accs = []
  new_accs = []
  count = 0

  for uid in chosen:
    if uid < 30000:
      network = searchspace.get_network(uid)
      # if not args.dataset == 'cifar10' or args.trainval:
      new_accs.append(searchspace.get_final_accuracy(uid, acc_type, False))
      val_accs.append(searchspace.get_final_accuracy(uid, val_acc_type, args.trainval))
    else:
      count = count + 1

  avg_acc = round(np.mean(accs), num_precision)
  variance_acc = round(np.var(accs), num_precision)

  avg_val_acc = round(np.mean(val_accs), num_precision)
  variance_val_acc = round(np.var(val_accs), num_precision)

  avg_new_acc = round(np.mean(new_accs), num_precision)
  variance_new_acc = round(np.var(new_accs), num_precision)
  
  avg_time = round(np.mean(times), num_precision)
  variance_time = round(np.var(times), num_precision)
  median_time =  round(np.median(times), num_precision)
  
  tau, p = stats.kendalltau(accs, scores)
  # new_tau, p = stats.kendalltau(new_accs, scores)

  acc_avg_var_str = f"{avg_acc}±{variance_acc}"
  val_acc_avg_var_str = f"{avg_val_acc}±{variance_val_acc}"
  new_acc_avg_var_str = f"{avg_new_acc}±{variance_new_acc}"

  avg_var_str = f"{avg_time}±{variance_time}"
  avg_str = f"{avg_time}"
  median_str = f"{median_time}"
  tau_str = f"{round(tau, num_precision)}"
  # new_tau_str = f"{round(new_tau, num_precision)}"
  
  df = pd.DataFrame({'Average +- Variance (s)' : avg_var_str,
                    'Avg Time (s)' : avg_str,
                    'Median Time (s)' : median_str,
                    'Accuracy (%)' : acc_avg_var_str,
                    'Val Accuracy (%)' : val_acc_avg_var_str,
                    'New Accuracy (%)' : new_acc_avg_var_str,
                    'Kendal Tau' : tau_str,
                    # 'New Kendal Tau' : new_tau_str
                     }, index=[0])

  df.to_csv(f"/content/drive/MyDrive/Mestrado/2022/results/{month}/{file}", sep='\t', encoding='utf-8')
  df2 = pd.concat({f"{file}": df})
  df_test = pd.concat([df_test, df2])

df_test.to_csv(f"/content/drive/MyDrive/Mestrado/2022/results/{month}/result.csv", sep='\t', encoding='utf-8')
print(count)

/usr/local/lib/python3.7/dist-packages/numpy/core/fromnumeric.py:3441: RuntimeWarning: Mean of empty slice.
  The average squared deviation is normally calculated as
/usr/local/lib/python3.7/dist-packages/numpy/core/_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  # Compute the mean.
/usr/local/lib/python3.7/dist-packages/numpy/core/fromnumeric.py:3724: RuntimeWarning: Degrees of freedom <= 0 for slice
/usr/local/lib/python3.7/dist-packages/numpy/core/_methods.py:223: RuntimeWarning: invalid value encountered in true_divide
  ret = um.true_divide(
/usr/local/lib/python3.7/dist-packages/numpy/core/_methods.py:254: RuntimeWarning: invalid value encountered in double_scalars
  ctx = contextlib_nullcontext(file)


0
